# TODO

link to LLM :
- write natural language question
- ask LLM to generate query
- otherwise, write Cypher working queries
- feed LLM working query to generate initial natural language question

same for supply chain and London tube :
- get info on each station

In [1]:
%load_ext autoreload
%autoreload 2

import os
import pandas as pd

import sys
sys.path.insert(1, f"{os.getenv('HOME')}/turingdb-examples/examples/functions")

from functions import *

In [2]:
path_data = f"{os.getenv('HOME')}/turingdb-examples/examples/data"
folder = "clinical_healthcare"

# healthcare_dataset - https://www.kaggle.com/datasets/prasad22/healthcare-dataset

In [3]:
df = pd.read_csv(f"{path_data}/{folder}/healthcare_dataset.csv")
df['Name'] = df['Name'].str.capitalize()
df = create_ID_column(df)
#df = df.replace('-', ' ', regex=True)

df = df.iloc[:10,:]

df

,Patient ID,Name,Age,Gender,Blood Type,Medical Condition,Date of Admission,Doctor,Hospital,Insurance Provider,Billing Amount,Room Number,Admission Type,Discharge Date,Medication,Test Results
0,00000,Bobby jackson,30,Male,B-,Cancer,2024-01-31,Matthew Smith,Sons and Miller,Blue Cross,18856.281306,328,Urgent,2024-02-02,Paracetamol,Normal
1,00001,Leslie terry,62,Male,A+,Obesity,2019-08-20,Samantha Davies,Kim Inc,Medicare,33643.327287,265,Emergency,2019-08-26,Ibuprofen,Inconclusive
2,00002,Danny smith,76,Female,A-,Obesity,2022-09-22,Tiffany Mitchell,Cook PLC,Aetna,27955.096079,205,Emergency,2022-10-07,Aspirin,Normal
3,00003,Andrew watts,28,Female,O+,Diabetes,2020-11-18,Kevin Wells,"Hernandez Rogers and Vang,",Medicare,37909.782410,450,Elective,2020-12-18,Ibuprofen,Abnormal
4,00004,Adrienne bell,43,Female,AB+,Cancer,2022-09-19,Kathleen Hanna,White-White,Aetna,14238.317814,458,Urgent,2022-10-09,Penicillin,Abnormal
5,00005,Emily johnson,36,Male,A+,Asthma,2023-12-20,Taylor Newton,Nunez-Humphrey,UnitedHealthcare,48145.110951,389,Urgent,2023-12-24,Ibuprofen,Normal
6,00006,Edward edwards,21,Female,AB-,Diabetes,2020-11-03,Kelly Olson,Group Middleton,Medicare,19580.872345,389,Emergency,2020-11-15,Paracetamol,Inconclusive
7,00007,Christina martinez,20,Female,A+,Cancer,2021-12-28,Suzanne Thomas,"Powell Robinson and Valdez,",Cigna,45820.462722,277,Emergency,2022-01-07,Paracetamol,Inconclusive
8,00008,Jasmine aguilar,82,Male,AB+,Asthma,2020-07-01,Daniel Ferguson,Sons Rich and,Cigna,50119.222792,316,Elective,2020-07-14,Aspirin,Abnormal
9,00009,Christopher berg,58,Female,AB-,Cancer,2021-05-23,Heather Day,Padilla-Walker,UnitedHealthcare,19784.631062,249,Elective,2021-06-22,Paracetamol,Inconclusive


In [4]:
label_str = 'displayName'

G = create_graph_from_df(
    df,
    directed=True,
    
    source_node_col={
        'id': 'Patient ID',
        label_str: 'Name',
        'type': 'Patient'
    },
    #target_node_col={
    #    'id': '',
    #    label_str: '',
    #    'type': ''
    #},
    attributes_source_node_cols=['Age', 'Date of Admission', 'Discharge Date'],
    #attributes_target_node_cols=None,
    optional_nodes_cols={
        'Gender': {
            'id': 'Gender',
            #label_str: '',
            #'attributes': '',
            'link_to_source': True,
            #'link_to_target': False,
            #'edge_attributes': ['']
        },
        'Blood Type': {
            'id': 'Blood Type',
            'link_to_source': True
        },
        'Medical Condition': {
            'id': 'Medical Condition',
            'link_to_source': True
        },
        'Doctor': {
            'id': 'Doctor',
            'link_to_source': True
        },
        'Hospital': {
            'id': 'Hospital',
            'attributes': ['Room Number'],
            'link_to_source': True
        },
        'Insurance Provider': {
            'id': 'Insurance Provider',
            'attributes': ['Billing Amount'],
            'link_to_source': True
        },
        'Admission Type': {
            'id': 'Admission Type',
            'link_to_source': True
        },
        'Medication': {
            'id': 'Medication',
            'link_to_source': True
        },
        'Test Results': {
            'id': 'Test Results',
            'link_to_source': True
        }
    },
    #attributes_edges=[''],
    #edge_type_col=''
)
print(f"Resulting graph : {G}")

Resulting graph : DiGraph with 57 nodes and 90 edges


In [5]:
for node in G.nodes(data=True):
    print(node)

('00000', {'displayName': 'Bobby jackson', 'type': 'Patient', 'Age': 30, 'Date of Admission': '2024-01-31', 'Discharge Date': '2024-02-02'})
('Male', {'displayName': 'Male'})
('B-', {'displayName': 'B-'})
('Cancer', {'displayName': 'Cancer'})
('Matthew Smith', {'displayName': 'Matthew Smith'})
('Sons and Miller', {'displayName': 'Sons and Miller', 'Room Number': 328})
('Blue Cross', {'displayName': 'Blue Cross', 'Billing Amount': 18856.281305978155})
('Urgent', {'displayName': 'Urgent'})
('Paracetamol', {'displayName': 'Paracetamol'})
('Normal', {'displayName': 'Normal'})
('00001', {'displayName': 'Leslie terry', 'type': 'Patient', 'Age': 62, 'Date of Admission': '2019-08-20', 'Discharge Date': '2019-08-26'})
('A+', {'displayName': 'A+'})
('Obesity', {'displayName': 'Obesity'})
('Samantha Davies', {'displayName': 'Samantha Davies'})
('Kim Inc', {'displayName': 'Kim Inc', 'Room Number': 265})
('Medicare', {'displayName': 'Medicare', 'Billing Amount': 33643.327286577885})
('Emergency', {

In [12]:
for edge in G.edges(data=True):
    print(edge)

('00000', 'Male', {})
('00000', 'B-', {})
('00000', 'Cancer', {})
('00000', 'Matthew Smith', {})
('00000', 'Sons and Miller', {})
('00000', 'Blue Cross', {})
('00000', 'Urgent', {})
('00000', 'Paracetamol', {})
('00000', 'Normal', {})
('00001', 'Male', {})
('00001', 'A+', {})
('00001', 'Obesity', {})
('00001', 'Samantha Davies', {})
('00001', 'Kim Inc', {})
('00001', 'Medicare', {})
('00001', 'Emergency', {})
('00001', 'Ibuprofen', {})
('00001', 'Inconclusive', {})
('00002', 'Female', {})
('00002', 'A-', {})
('00002', 'Obesity', {})
('00002', 'Tiffany Mitchell', {})
('00002', 'Cook PLC', {})
('00002', 'Aetna', {})
('00002', 'Emergency', {})
('00002', 'Aspirin', {})
('00002', 'Normal', {})
('00003', 'Female', {})
('00003', 'O+', {})
('00003', 'Diabetes', {})
('00003', 'Kevin Wells', {})
('00003', 'Hernandez Rogers and Vang,', {})
('00003', 'Medicare', {})
('00003', 'Elective', {})
('00003', 'Ibuprofen', {})
('00003', 'Abnormal', {})
('00004', 'Female', {})
('00004', 'AB+', {})
('00004',

In [7]:
from turingdb import TuringDB

# Create TuringDB client
client = TuringDB(host="http://localhost:6666")

# Create a new graph
graph_name = "patient_network"
client.query(f"CREATE GRAPH {graph_name}")
client.set_graph(graph_name)

# Create a new change on the graph
change = client.query("CHANGE NEW")["Change ID"][0]

# Checkout into the change
client.checkout(change=change)

# Build CREATE command from networkx object
create_command = build_create_command_from_networkx(G)
#print(create_command)

# Run CREATE command
client.query(create_command)

# Commit the change
client.query("COMMIT")
client.query("CHANGE SUBMIT")

""


In [10]:
show_link(link_str="https://8080--main--newmain--maximevincent.work.turing.bio/")

# BRCA - https://www.kaggle.com/datasets/amandam1/breastcancerdataset

In [34]:
df = pd.read_csv(f"{path_data}/{folder}/BRCA.csv")
df = df.dropna(axis=0, how='all')

df = df.iloc[:10,:]

df

,Patient_ID,Age,Gender,Protein1,Protein2,Protein3,Protein4,Tumour_Stage,Histology,ER status,PR status,HER2 status,Surgery_type,Date_of_Surgery,Date_of_Last_Visit,Patient_Status
0,TCGA-D8-A1XD,36.0,FEMALE,0.080353,0.42638,0.547150,0.273680,III,Infiltrating Ductal Carcinoma,Positive,Positive,Negative,Modified Radical Mastectomy,15-Jan-17,19-Jun-17,Alive
1,TCGA-EW-A1OX,43.0,FEMALE,-0.420320,0.57807,0.614470,-0.031505,II,Mucinous Carcinoma,Positive,Positive,Negative,Lumpectomy,26-Apr-17,09-Nov-18,Dead
2,TCGA-A8-A079,69.0,FEMALE,0.213980,1.31140,-0.327470,-0.234260,III,Infiltrating Ductal Carcinoma,Positive,Positive,Negative,Other,08-Sep-17,09-Jun-18,Alive
3,TCGA-D8-A1XR,56.0,FEMALE,0.345090,-0.21147,-0.193040,0.124270,II,Infiltrating Ductal Carcinoma,Positive,Positive,Negative,Modified Radical Mastectomy,25-Jan-17,12-Jul-17,Alive
4,TCGA-BH-A0BF,56.0,FEMALE,0.221550,1.90680,0.520450,-0.311990,II,Infiltrating Ductal Carcinoma,Positive,Positive,Negative,Other,06-May-17,27-Jun-19,Dead
5,TCGA-AO-A1KQ,84.0,MALE,-0.081872,1.72410,-0.057335,0.043025,III,Infiltrating Ductal Carcinoma,Positive,Positive,Negative,Modified Radical Mastectomy,18-Sep-17,15-Nov-21,Alive
6,TCGA-D8-A73X,53.0,FEMALE,-0.069535,1.41830,-0.361050,0.391580,II,Infiltrating Ductal Carcinoma,Positive,Positive,Negative,Simple Mastectomy,04-Feb-17,07-Feb-18,Alive
7,TCGA-A7-A426,50.0,FEMALE,0.672490,1.27900,-0.321070,-0.112390,III,Infiltrating Ductal Carcinoma,Positive,Positive,Negative,Simple Mastectomy,16-May-17,NaN,NaN
8,TCGA-EW-A1P5,77.0,FEMALE,-0.151750,-0.66332,1.189400,0.217180,II,Infiltrating Ductal Carcinoma,Positive,Positive,Negative,Modified Radical Mastectomy,28-Sep-17,28-Sep-18,Alive
9,TCGA-A8-A09A,40.0,FEMALE,-0.565700,1.26680,-0.293460,0.193950,II,Infiltrating Lobular Carcinoma,Positive,Positive,Positive,Other,14-Feb-17,15-Dec-17,Alive


In [35]:
label_str = 'displayName'

G = create_graph_from_df(
    df,
    directed=True,
    
    source_node_col={
        'id': 'Patient_ID',
        #label_str: 'Name',
        'type': 'Patient'
    },
    #target_node_col={
    #    'id': '',
    #    label_str: '',
    #    'type': ''
    #},
    attributes_source_node_cols=['Age', 'Protein1', 'Protein2', 'Protein3',	'Protein4'],
    #attributes_target_node_cols=None,
    optional_nodes_cols={
        'Gender': {
            'id': 'Gender',
            #label_str: '',
            #'attributes': '',
            'link_to_source': True,
            #'link_to_target': False,
            #'edge_attributes': ['']
        },
        'Histology': {
            'id': 'Histology',
            'link_to_source': True,
            'edge_attributes': ['Tumour_Stage', 'ER status', 'PR status', 'HER2 status']
        },
        'Surgery_type': {
            'id': 'Surgery_type',
            'link_to_source': True,
            'edge_attributes': ['Date_of_Surgery', 'Date_of_Last_Visit']
        },
        'Patient_Status': {
            'id': 'Patient_Status',
            'link_to_source': True
        },
    },
    #attributes_edges=[''],
    #edge_type_col=''
)
print(f"Resulting graph : {G}")

Resulting graph : DiGraph with 22 nodes and 40 edges


In [36]:
n_first = 10

print("NODES :")
for node in list(G.nodes(data=True))[:n_first]:
    print(node)
print()

print("EDGES :")
for edge in list(G.edges(data=True))[:n_first]:
    print(edge)

NODES :
('TCGA-D8-A1XD', {'displayName': 'TCGA-D8-A1XD', 'type': 'Patient', 'Age': 36.0, 'Protein1': 0.080353, 'Protein2': 0.42638, 'Protein3': 0.54715, 'Protein4': 0.27368})
('FEMALE', {'displayName': 'FEMALE'})
('Infiltrating Ductal Carcinoma', {'displayName': 'Infiltrating Ductal Carcinoma'})
('Modified Radical Mastectomy', {'displayName': 'Modified Radical Mastectomy'})
('Alive', {'displayName': 'Alive'})
('TCGA-EW-A1OX', {'displayName': 'TCGA-EW-A1OX', 'type': 'Patient', 'Age': 43.0, 'Protein1': -0.42032, 'Protein2': 0.57807, 'Protein3': 0.61447, 'Protein4': -0.031505})
('Mucinous Carcinoma', {'displayName': 'Mucinous Carcinoma'})
('Lumpectomy', {'displayName': 'Lumpectomy'})
('Dead', {'displayName': 'Dead'})
('TCGA-A8-A079', {'displayName': 'TCGA-A8-A079', 'type': 'Patient', 'Age': 69.0, 'Protein1': 0.21398, 'Protein2': 1.3114, 'Protein3': -0.32747, 'Protein4': -0.23426})

EDGES :
('TCGA-D8-A1XD', 'FEMALE', {})
('TCGA-D8-A1XD', 'Infiltrating Ductal Carcinoma', {'Tumour_Stage': 'I

In [37]:
graph_name = "BRCA_dataset2"

In [38]:
from turingdb import TuringDB

# Create TuringDB client
client = TuringDB(host="http://localhost:6666")

# Create a new graph
client.query(f"CREATE GRAPH {graph_name}")
client.set_graph(graph_name)

# Create a new change on the graph
change = client.query("CHANGE NEW")["Change ID"][0]

# Checkout into the change
client.checkout(change=change)

# Build CREATE command from networkx object
create_command = build_create_command_from_networkx(G)
print(create_command)

# Run CREATE command
client.query(create_command)

# Commit the change
client.query("COMMIT")
client.query("CHANGE SUBMIT")

CREATE (n0:Patient {id:"TCGA-D8-A1XD", "displayName":"TCGA-D8-A1XD", "type":"Patient", "Age":"36.0", "Protein1":"0.080353", "Protein2":"0.42638", "Protein3":"0.54715", "Protein4":"0.27368"}),
(n1:Node {id:"FEMALE", "displayName":"FEMALE"}),
(n2:Node {id:"Infiltrating Ductal Carcinoma", "displayName":"Infiltrating Ductal Carcinoma"}),
(n3:Node {id:"Modified Radical Mastectomy", "displayName":"Modified Radical Mastectomy"}),
(n4:Node {id:"Alive", "displayName":"Alive"}),
(n5:Patient {id:"TCGA-EW-A1OX", "displayName":"TCGA-EW-A1OX", "type":"Patient", "Age":"43.0", "Protein1":"-0.42032", "Protein2":"0.57807", "Protein3":"0.61447", "Protein4":"-0.031505"}),
(n6:Node {id:"Mucinous Carcinoma", "displayName":"Mucinous Carcinoma"}),
(n7:Node {id:"Lumpectomy", "displayName":"Lumpectomy"}),
(n8:Node {id:"Dead", "displayName":"Dead"}),
(n9:Patient {id:"TCGA-A8-A079", "displayName":"TCGA-A8-A079", "type":"Patient", "Age":"69.0", "Protein1":"0.21398", "Protein2":"1.3114", "Protein3":"-0.32747", "Pro

""


In [39]:
show_link(link_str="https://8080--main--newmain--maximevincent.work.turing.bio/")

In [46]:
#separate_create_commands = [('CREATE ' if not command.startswith('CREATE') else '') + command for command in create_command.split('\n')]
#
#for command in separate_create_commands:
#    print(command)
#    client.query(command)

# Covid_Data - https://www.kaggle.com/datasets/meirnizri/covid19-dataset

In [16]:
df = pd.read_csv(f"{path_data}/{folder}/Covid_Data.csv")
#df = df.dropna(axis=0, how='all')
df

,USMER,MEDICAL_UNIT,SEX,PATIENT_TYPE,DATE_DIED,INTUBED,PNEUMONIA,AGE,PREGNANT,DIABETES,...,ASTHMA,INMSUPR,HIPERTENSION,OTHER_DISEASE,CARDIOVASCULAR,OBESITY,RENAL_CHRONIC,TOBACCO,CLASIFFICATION_FINAL,ICU
0,2,1,1,1,03/05/2020,97,1,65,2,2,...,2,2,1,2,2,2,2,2,3,97
1,2,1,2,1,03/06/2020,97,1,72,97,2,...,2,2,1,2,2,1,1,2,5,97
2,2,1,2,2,09/06/2020,1,2,55,97,1,...,2,2,2,2,2,2,2,2,3,2
3,2,1,1,1,12/06/2020,97,2,53,2,2,...,2,2,2,2,2,2,2,2,7,97
4,2,1,2,1,21/06/2020,97,2,68,97,1,...,2,2,1,2,2,2,2,2,3,97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,2,13,2,1,9999-99-99,97,2,40,97,2,...,2,2,2,2,2,2,2,2,7,97
1048571,1,13,2,2,9999-99-99,2,2,51,97,2,...,2,2,1,2,2,2,2,2,7,2
1048572,2,13,2,1,9999-99-99,97,2,55,97,2,...,2,2,2,2,2,2,2,2,7,97
1048573,2,13,2,1,9999-99-99,97,2,28,97,2,...,2,2,2,2,2,2,2,2,7,97


# Disease_symptom_and_patient_profile_dataset - https://www.kaggle.com/datasets/uom190346a/disease-symptoms-and-patient-profile-dataset

In [3]:
df = pd.read_csv(f"{path_data}/{folder}/Disease_symptom_and_patient_profile_dataset.csv")
df = create_ID_column(df)

df = df.iloc[:10,:]

df

,Patient ID,Disease,Fever,Cough,Fatigue,Difficulty Breathing,Age,Gender,Blood Pressure,Cholesterol Level,Outcome Variable
0,000,Influenza,Yes,No,Yes,Yes,19,Female,Low,Normal,Positive
1,001,Common Cold,No,Yes,Yes,No,25,Female,Normal,Normal,Negative
2,002,Eczema,No,Yes,Yes,No,25,Female,Normal,Normal,Negative
3,003,Asthma,Yes,Yes,No,Yes,25,Male,Normal,Normal,Positive
4,004,Asthma,Yes,Yes,No,Yes,25,Male,Normal,Normal,Positive
...,...,...,...,...,...,...,...,...,...,...,...
344,344,Stroke,Yes,No,Yes,No,80,Female,High,High,Positive
345,345,Stroke,Yes,No,Yes,No,85,Male,High,High,Positive
346,346,Stroke,Yes,No,Yes,No,85,Male,High,High,Positive
347,347,Stroke,Yes,No,Yes,No,90,Female,High,High,Positive


In [10]:
label_str = 'displayName'

G = create_graph_from_df(
    df,
    directed=True,
    
    source_node_col={
        'id': 'Patient ID',
        'type': 'Patient'
    },
    attributes_source_node_cols=['Age'],
    
    optional_nodes_cols={
        'Disease': {'link_to_source': True},
        'Gender': {'link_to_source': True},
        'Fever': {'link_to_source': True},
        'Cough': {'link_to_source': True},
        'Fatigue': {'link_to_source': True},
        'Difficulty Breathing': {'link_to_source': True},
        'Blood Pressure': {'link_to_source': True},
        'Cholesterol Level': {'link_to_source': True},
        'Outcome Variable': {'link_to_source': True}
    }
)
print(f"Resulting graph : {G}")

Resulting graph : DiGraph with 474 nodes and 2202 edges


In [11]:
n_first = 10

print("NODES :")
for node in list(G.nodes(data=True))[:n_first]:
    print(node)
print()

print("EDGES :")
for edge in list(G.edges(data=True))[:n_first]:
    print(edge)

NODES :
('000', {'displayName': '000', 'type': 'Patient', 'Age': 19})
('Influenza', {'displayName': 'Influenza'})
('Female', {'displayName': 'Female'})
('Yes', {'displayName': 'Yes'})
('No', {'displayName': 'No'})
('Low', {'displayName': 'Low'})
('Normal', {'displayName': 'Normal'})
('Positive', {'displayName': 'Positive'})
('001', {'displayName': '001', 'type': 'Patient', 'Age': 25})
('Common Cold', {'displayName': 'Common Cold'})

EDGES :
('000', 'Influenza', {})
('000', 'Female', {})
('000', 'Yes', {})
('000', 'No', {})
('000', 'Low', {})
('000', 'Normal', {})
('000', 'Positive', {})
('001', 'Common Cold', {})
('001', 'Female', {})
('001', 'No', {})


In [13]:
from turingdb import TuringDB

# Create TuringDB client
client = TuringDB(host="http://localhost:6666")

# Create a new graph
graph_name = "disease_symptom"
client.query(f"CREATE GRAPH {graph_name}")
client.set_graph(graph_name)

# Create a new change on the graph
change = client.query("CHANGE NEW")["Change ID"][0]

# Checkout into the change
client.checkout(change=change)

# Build CREATE command from networkx object
create_command = build_create_command_from_networkx(G)
#print(create_command)

# Run CREATE command
client.query(create_command)

# Commit the change
client.query("COMMIT")
client.query("CHANGE SUBMIT")

""


In [14]:
show_link(link_str="https://8080--main--newmain--maximevincent.work.turing.bio/")

# heart_attack_prediction_dataset - https://www.kaggle.com/datasets/iamsouravbanerjee/heart-attack-prediction-dataset

In [18]:
pd.set_option('display.max_columns', None)

In [19]:
df = pd.read_csv(f"{path_data}/{folder}/heart_attack_prediction_dataset.csv")

df = df.iloc[:10,:]

df

,Patient ID,Age,Sex,Cholesterol,Blood Pressure,Heart Rate,Diabetes,Family History,Smoking,Obesity,Alcohol Consumption,Exercise Hours Per Week,Diet,Previous Heart Problems,Medication Use,Stress Level,Sedentary Hours Per Day,Income,BMI,Triglycerides,Physical Activity Days Per Week,Sleep Hours Per Day,Country,Continent,Hemisphere,Heart Attack Risk
0,BMW7812,67,Male,208,158/88,72,0,0,1,0,0,4.168189,Average,0,0,9,6.615001,261404,31.251233,286,0,6,Argentina,South America,Southern Hemisphere,0
1,CZE1114,21,Male,389,165/93,98,1,1,1,1,1,1.813242,Unhealthy,1,0,1,4.963459,285768,27.194973,235,1,7,Canada,North America,Northern Hemisphere,0
2,BNI9906,21,Female,324,174/99,72,1,0,0,0,0,2.078353,Healthy,1,1,9,9.463426,235282,28.176571,587,4,4,France,Europe,Northern Hemisphere,0
3,JLN3497,84,Male,383,163/100,73,1,1,1,0,1,9.828130,Average,1,0,9,7.648981,125640,36.464704,378,3,4,Canada,North America,Northern Hemisphere,0
4,GFO8847,66,Male,318,91/88,93,1,1,1,1,0,5.804299,Unhealthy,1,0,6,1.514821,160555,21.809144,231,1,5,Thailand,Asia,Northern Hemisphere,0
5,ZOO7941,54,Female,297,172/86,48,1,1,1,0,1,0.625008,Unhealthy,1,1,2,7.798752,241339,20.146840,795,5,10,Germany,Europe,Northern Hemisphere,1
6,WYV0966,90,Male,358,102/73,84,0,0,1,0,1,4.098177,Healthy,0,0,7,0.627356,190450,28.885811,284,4,10,Canada,North America,Northern Hemisphere,1
7,XXM0972,84,Male,220,131/68,107,0,0,1,1,1,3.427929,Average,0,1,4,10.543780,122093,22.221862,370,6,7,Japan,Asia,Northern Hemisphere,1
8,XCQ5937,20,Male,145,144/105,68,1,0,1,1,0,16.868302,Average,0,0,5,11.348787,25086,35.809901,790,7,4,Brazil,South America,Southern Hemisphere,0
9,FTJ5456,43,Female,248,160/70,55,0,1,1,1,1,0.194515,Unhealthy,0,0,4,4.055115,209703,22.558917,232,7,7,Japan,Asia,Northern Hemisphere,0


In [28]:
label_str = 'displayName'

G = create_graph_from_df(
    df,
    directed=True,
    
    source_node_col={
        'id': 'Patient ID',
        'type': 'Patient'
    },
    attributes_source_node_cols=['Age',
        'Cholesterol',
        'Blood Pressure',
        'Heart Rate',
        'Exercise Hours Per Week',
        'Sedentary Hours Per Day',
        'Income',
        'BMI',
        'Triglycerides'
    ],
    
    optional_nodes_cols={
        col: {'link_to_source': True}
        for col in
        [
            'Diabetes',
            'Family History',
            'Smoking',
            'Obesity',
            'Alcohol Consumption',
            'Previous Heart Problems',
            'Medication Use',
            'Stress Level',
            'Physical Activity Days Per Week',
            'Sleep Hours Per Day',
            'Country',
            'Continent',
            'Hemisphere',
            'Heart Attack Risk'
        ]
    }
)
print(f"Resulting graph : {G}")

Resulting graph : DiGraph with 33 nodes and 74 edges


In [29]:
n_first = 10

print("NODES :")
for node in list(G.nodes(data=True))[:n_first]:
    print(node)
print()

print("EDGES :")
for edge in list(G.edges(data=True))[:n_first]:
    print(edge)

NODES :
('BMW7812', {'displayName': 'BMW7812', 'type': 'Patient', 'Age': 67, 'Cholesterol': 208, 'Blood Pressure': '158/88', 'Heart Rate': 72, 'Exercise Hours Per Week': 4.168188835442079, 'Sedentary Hours Per Day': 6.61500145291406, 'Income': 261404, 'BMI': 31.2512327252954, 'Triglycerides': 286})
(0, {'displayName': '0'})
(1, {'displayName': '1'})
(9, {'displayName': '9'})
(6, {'displayName': '6'})
('Argentina', {'displayName': 'Argentina'})
('South America', {'displayName': 'South America'})
('Southern Hemisphere', {'displayName': 'Southern Hemisphere'})
('CZE1114', {'displayName': 'CZE1114', 'type': 'Patient', 'Age': 21, 'Cholesterol': 389, 'Blood Pressure': '165/93', 'Heart Rate': 98, 'Exercise Hours Per Week': 1.8132416178634456, 'Sedentary Hours Per Day': 4.963458839757678, 'Income': 285768, 'BMI': 27.1949733519874, 'Triglycerides': 235})
(7, {'displayName': '7'})

EDGES :
('BMW7812', 0, {})
('BMW7812', 1, {})
('BMW7812', 9, {})
('BMW7812', 6, {})
('BMW7812', 'Argentina', {})
('

In [30]:
from turingdb import TuringDB

# Create TuringDB client
client = TuringDB(host="http://localhost:6666")

# Create a new graph
graph_name = "heart_attack_prediction_dataset"
client.query(f"CREATE GRAPH {graph_name}")
client.set_graph(graph_name)

# Create a new change on the graph
change = client.query("CHANGE NEW")["Change ID"][0]

# Checkout into the change
client.checkout(change=change)

# Build CREATE command from networkx object
create_command = build_create_command_from_networkx(G)
#print(create_command)

# Run CREATE command
client.query(create_command)

# Commit the change
client.query("COMMIT")
client.query("CHANGE SUBMIT")

""


In [31]:
show_link(link_str="https://8080--main--newmain--maximevincent.work.turing.bio/")